In [22]:
from datasets import load_wine
from tools import *
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import plotly.express as px
import pandas as pd
from sklearn.model_selection import cross_val_score
import numpy as np
np.set_printoptions(suppress=True)

In [23]:
df_original, df_train, df_test = load_wine()
df_train.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,target
779,7.1,0.52,0.03,2.6,0.076,21.0,92.0,0.99745,3.50,0.60,9.8,5
185,8.9,0.31,0.57,2.0,0.111,26.0,85.0,0.99710,3.26,0.53,9.7,5
309,7.4,0.29,0.38,1.7,0.062,9.0,30.0,0.99680,3.41,0.53,9.5,6
1513,6.4,0.56,0.15,1.8,0.078,17.0,65.0,0.99294,3.33,0.60,10.5,6
1185,7.0,0.43,0.30,2.0,0.085,6.0,39.0,0.99346,3.33,0.46,11.9,6


In [24]:
train_X = df_train.drop(['target'], axis=1)
train_y = df_train['target']
test_X = df_test.drop(['target'], axis=1)
test_y = df_test['target']
features = train_X.columns

In [25]:
# scaler = StandardScaler().fit(train_X)
# train_X = pd.DataFrame(scaler.transform(train_X), columns=features)
# test_X = pd.DataFrame(scaler.transform(test_X), columns=features)

## Pocet komponent

In [26]:
pca = PCA(n_components=11)
pca = pca.fit(train_X)

In [27]:
# vidime ze nejvice variance popisuji prvni 3-4 komponenty, zvolime 3 pro lepsi vizualizaci
pca.explained_variance_ratio_

array([0.94775399, 0.04711021, 0.00266126, 0.00153144, 0.00086973,
       0.0000353 , 0.00001911, 0.00000959, 0.00000815, 0.00000121,
       0.        ])

## Regresion

In [28]:
pca = PCA(n_components=3)
pca = pca.fit(train_X)

In [29]:
transformed_train_X = pca.transform(train_X)
transformed_test_X = pca.transform(test_X)

In [30]:
transformed_df = pd.DataFrame(transformed_train_X, columns=['x','y','z'])
transformed_df['target'] = train_y.values
px.scatter_3d(transformed_df, x='x', y='y', z='z',
              color='target', size='target')

In [31]:
cross_val_score(LinearRegression(), transformed_train_X, train_y).mean()

0.03386877798618557

In [32]:
model = LinearRegression()
model = model.fit(transformed_train_X, train_y)

In [33]:
model_performance(model, transformed_train_X, train_y, transformed_test_X, test_y)

Train score
0.049180359677305496
Test score
0.08305734746670868


## Overeni vyberu komponent

In [34]:
scores = []
components = []
for i in range(1, len(df_train.columns) - 1):
    pca = PCA(n_components=i)
    pca = pca.fit(train_X)
    transformed_train_X = pca.transform(train_X)
    transformed_test_X = pca.transform(test_X)
    components.append(i)
    scores.append(cross_val_score(LinearRegression(), transformed_train_X, train_y).mean())

pd.DataFrame(zip(components, scores), columns=['components', 'score'])

,components,score
0,1,0.022509
1,2,0.024533
2,3,0.033869
3,4,0.043272
4,5,0.222173
5,6,0.311800
6,7,0.310894
7,8,0.312429
8,9,0.334423
9,10,0.342725
